# Tutorial 9: Retrieval-augmented generation (RAG) y Agentes.

### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Felipe Villena](https://fabianvillena.cl/).
- Profesor Auxiliar: María José Zambrano


### Objetivos del Tutorial
- Introducir Langchain
- Qué es RAG y como utilizarlo
- Comprender los que son los agentes y como utilizarlos

Tutorial basado en clase del curso [Laboratorio de Programación Científica para Ciencia de Datos.](https://github.com/MDS7202/MDS7202/tree/main)

### **Google AI Studio**

Usaremos `Google AI Studio` para habilitar el uso de LLMs y Embeddings de Google. Simplemente deben registrarse con su cuenta google y obtener su API KEY desde el siguiente enlace: [Google AI Studio](https://aistudio.google.com/app/u/1/apikey).

### **Tavily**

En paralelo, utilizaremos `Tavily` como motor de búsqueda para potenciar las respuestas de nuestros agentes. Tal como en el paso anterior, solo deben registrarse y obtener su API KEY desde el siguiente enlace: [Tavily](https://tavily.com/).

### **Configurar credenciales en ambiente**

Una vez se tienen todas las credenciales, pasamos a activarlas en nuestro ambiente local por medio del siguiente código:

### **Configurar credenciales en ambiente**

Una vez se tienen todas las credenciales, pasamos a activarlas en nuestro ambiente local por medio del siguiente código:

In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Google AI API key: ··········
Enter your Tavily API key: ··········


In [2]:
%pip install --upgrade --quiet  langchain-google-genai
%pip install --quiet --upgrade langchain-text-splitters langchain-community
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.8 MB/s eta 0:00:00


## **LangChain 🦜**

`LangChain` es un framework de código abierto diseñado para desarrollar aplicaciones basadas en LLM. Basa su funcionamiento en los siguientes componentes:

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2024-01//LLM/langchain.png' width=450  />
</center>

**1. Prompts y Plantillas de Prompts**: Los prompts son consultas que enviamos a los LLMs, y su calidad impacta las respuestas. LangChain facilita la creación y gestión de prompts mediante plantillas que combinan instrucciones, contenido y consultas.

```python
template = """
You are required to answer the following question in form of bullet points based on the provided context.
The answer should be answer as a doctor and your language is spanish.:
{context}
Now based on above context answer the following question:
{question}

Answer:
"""
```

**2. Modelos**: LangChain no incluye LLMs, pero permite integrar fácilmente varios modelos de lenguaje (como GPT-3, BLOOM), de chat (como GPT-3.5-turbo) y de embedding de texto (de CohereAI, HuggingFace y OpenAI) mediante un simple framework donde solo necesitas la API Key o cargar el modelo en memoria.

```python
from langchain.llms import OpenAI

openai = OpenAI(
   openai_api_key=”YOUR OPEN AI API KEY”,
   model_name="gpt-3.5-turbo-16k",
)
```

**3. Chains**: LangChain permite crear flujos de trabajo, o "chains," que son secuencias de llamadas a modelos de lenguaje o a herramientas externas. Estos flujos pueden incluir varias etapas de procesamiento, donde la salida de una etapa se convierte en la entrada de la siguiente. son el simil de pipelines de `scikit-learn`.

```python
chain = prompt | llm | StrOutputParser()
```

**4. Memoria**: Por defecto, las cadenas en LangChain son sin estado, es decir, no guarda un registro de las interacciones anteriores hechas con el modelo. Para superar esto, LangChain nos asiste con *buffers* para almacenar de forma iterativa las interacciones realizadas.

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2024-01//LLM/memory.png' width=600 />
</center>

**5. Agentes**: Los agentes en LangChain toman decisiones sobre acciones según la entrada o el estado del flujo, como buscar información o llamar a una API, lo que permite crear aplicaciones interactivas y adaptables.

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2024-01//LLM/agents.png' width=450  />
</center>

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)

llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f02710aab00>, default_metadata=())

In [39]:
print('hola mundo')

hola mundo


In [4]:
question = "hola!"
response = llm.invoke(question) # invoke para interactuar con el modelo
response.content

'Hola! ¿Cómo estás?\n'

In [5]:
from langchain_core.prompts import PromptTemplate

# template: noten como la variable {question} va a ser completada con la pregunta del usuario.
template = '''
Eres un profesional experto en halterofilia
Tu único rol es responder de la forma más completa posible la pregunta del usuario.

Pregunta: {question}
Respuesta útil:
'''

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nEres un profesional experto en halterofilia\nTu único rol es responder de la forma más completa posible la pregunta del usuario.\n\nPregunta: {question}\nRespuesta útil:\n')

In [6]:
chain = prompt | llm # definimos la cadena
response = chain.invoke("hola!") # interactuamos con ella a través de invoke
response

AIMessage(content='Hola!  Como profesional experto en halterofilia, puedo ayudarte con una amplia gama de temas relacionados con este deporte. Para darte la respuesta más completa posible, necesito saber qué te interesa específicamente.  \n\nPor ejemplo, ¿te gustaría saber sobre:\n\n* **Técnicas de levantamiento:**  Como el arranque, el envión, o aspectos específicos de cada uno como la primera tracción, el tirón, el agarre, la recepción, etc.  Puedo explicarte la biomecánica, los puntos clave de cada fase, y cómo corregir errores comunes.\n\n* **Programación de entrenamiento:**  Diseño de programas para principiantes, intermedios o avanzados, incluyendo periodización, volumen, intensidad, frecuencia,  descanso y recuperación.  Puedo ayudarte a crear un plan personalizado basado en tus objetivos y nivel de experiencia.\n\n* **Nutrición para halterofilia:**  Dieta óptima para ganar masa muscular, fuerza y potencia, incluyendo macronutrientes, micronutrientes, hidratación y suplementació

In [7]:
response = chain.invoke("Cuales son la categorías?") # interactuamos con ella a través de invoke
response

AIMessage(content='Las categorías de halterofilia se definen por el peso corporal del atleta y varían ligeramente según la federación (principalmente la IWF - International Weightlifting Federation - y las federaciones nacionales).  No existe una única lista universal, ya que las categorías pueden ser ajustadas periódicamente por la IWF.  Sin embargo, puedo proporcionarte una descripción general de las categorías actuales (a fecha de respuesta, octubre de 2023) y cómo funcionan,  teniendo en cuenta que estas pueden sufrir modificaciones en el futuro.\n\n**Categorías Masculinas (IWF):**\n\nLa IWF actualmente utiliza las siguientes categorías de peso para hombres:\n\n* **55 kg:**  Hasta 55 kilogramos\n* **61 kg:**  Hasta 61 kilogramos\n* **67 kg:**  Hasta 67 kilogramos\n* **73 kg:**  Hasta 73 kilogramos\n* **81 kg:**  Hasta 81 kilogramos\n* **89 kg:**  Hasta 89 kilogramos\n* **96 kg:**  Hasta 96 kilogramos\n* **102 kg:** Hasta 102 kilogramos\n* **109 kg:** Hasta 109 kilogramos\n* **+109 

**¿Qué pasa si le preguntamos al llm sobre eventos que pasaron hace algunos años?**

In [8]:
response = llm.invoke("qué sabes del plebiscito constitucional del 2020 en Chile?")
print(response.content)

El plebiscito nacional de Chile de 2020, realizado el 25 de octubre, fue una consulta ciudadana para decidir si se debía o no redactar una nueva Constitución para reemplazar la Constitución de 1980, heredada de la dictadura de Augusto Pinochet.  La pregunta en la papeleta era: "¿Aprueba usted una nueva Constitución?".

Los resultados fueron un rotundo **Sí** a la redacción de una nueva Constitución.  Si bien la participación fue alta (50,9%),  el apoyo a la opción "Aprueba usted una nueva Constitución?" fue abrumador, con un **78,28% de votos a favor** y un 21,72% en contra.  Este resultado marcó un hito histórico, demostrando un amplio rechazo a la Constitución vigente y un deseo de cambio profundo en el sistema político chileno.

Sin embargo, el plebiscito de 2020 solo respondió a la primera pregunta sobre si se debía o no redactar una nueva Constitución.  Una segunda pregunta, sobre el mecanismo para redactar la nueva Constitución (Convención Constitucional o Asamblea Mixta Constitu

**Qué pasaría si le pregunto al LLM algúna pregunta de actualidad?** Por ejemplo:

In [9]:
response = llm.invoke("qué sabes sobre las elecciones municipales 2024 de Chile?")
print(response.content)

Las elecciones municipales de Chile en 2024 aún están bastante lejos, por lo que la información concreta es limitada.  No hay candidatos oficiales, ni campañas en marcha a gran escala.  Sin embargo, podemos hablar de lo que se sabe y se espera:

* **Fecha:**  Las elecciones municipales en Chile se realizan cada cuatro años.  La próxima elección se llevará a cabo en **octubre de 2024**, aunque la fecha exacta aún no se ha definido oficialmente por el Servel (Servicio Electoral).

* **Cargos a elegir:** Se elegirán alcaldes y concejales para cada comuna del país.

* **Contexto político:** Las elecciones se desarrollarán en un contexto político que probablemente estará marcado por:

    * **El gobierno en turno:** El resultado de las elecciones influirá en la gobernabilidad del gobierno de turno para el periodo 2022-2026.  La oposición buscará obtener mayorías en las comunas para contrarrestar el poder del gobierno.
    * **El nuevo proceso constituyente:** El resultado del nuevo proceso 

## **Retrieval Augmented Generation (RAG) 🔎📖**

**Retrieval Augmented Generation (RAG)** es una técnica para **ampliar el conocimiento** de los modelos de lenguaje grandes (LLMs) con **datos adicionales**, usualmente provenientes de **fuentes externas**. Si bien podemos hacer RAG con cualquier tipo de información externa, para esta sección nos concentraremos en hacer RAG sobre **documentos PDF**. En otras palabras, estaremos **nutriendo a nuestro LLM** con la información contenida en uno o más **documentos con extensión .pdf**.

<center>
<img src='https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2024-01//LLM/rag-framework.webp' width=450/>
</center>

### **¿Cómo funciona?**

La idea principal de esta solución es **responder la pregunta del usuario usando sólo los fragmentos de texto relevantes** para la pregunta en cuestión.

Con esto en consideración, para implementar una solución RAG sobre documentos PDF debemos pasar por 2 grandes etapas:

**1. Indexing**: Es el proceso por el cual **transformamos nuestros documentos** (.pdf) a **representaciones vectoriales**. Este es un paso que **se realiza sólo una vez**, pero fudamental para recuperar los fragmentos de texto relevantes.

 A su vez, para indexar un documento es necesario pasar por los siguientes pasos:
 - **Load**: **Ingesta del documento** al ambiente de trabajo (e.g: lectura y almacenamiento en strings).
 - **Split**: **Separación del documento en chunks de texto**. Chunks muy grandes pueden diluir la información necesaria para responder, chunks muy chicos pueden inducir a que se pierda el contexto.
 - **Embed**: Cada uno de los **chunks generados se transforman en representaciones vectoriales**, los cuales tienen la capacidad de almacenar la semántica, sintaxis y el contexto del chunk. Para esto, se utilizan modelos de lenguaje especializados en este tipo de transformaciones.
 - **Store**: Con los documentos vectorizados, **se almacenan estas representaciones** en una *vector store*.

<center>
<img src='https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png' width=700/>
</center>

**2. Retrieval and Generation**: Es el proceso en la que proporcionamos información al LLM para que responda de manera acorde. Consta de las siguientes etapas:
- **Retrieve**: Proceso por el cual **se buscan los *top k* chunks más relevantes a la pregunta**. Para lograr esto, se utilizan técnicas de recuperación de información.
- **Generate**: **Se entregan los *top k*** chunks al LLM y **se responde la pregunta** del usuario.

<center>
<img src='https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png' width=700/>
</center>

Ya que conocemos el funcionamiento básico de una solución RAG, pasemos a ilustrar esto con un ejemplo. Para esto, construiremos un RAG a partir del  [Informe de las Elecciones Municipales 2024](https://www.decidechile.cl/informes/informe-municipales-2024/) realizado por [Decide Chile](https://www.decidechile.cl/).

Primero instalamos las librerias necesarias:

In [10]:
%pip install --upgrade --quiet faiss-cpu langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.9 MB/s eta 0:00:00


### **Indexing**

Recordando los pasos descritos al principio de esta sección, la primera tarea para implementar una solución RAG es **indexar los documentos**.  

#### **Load**

Ya que nuestro documento es de extensión .pdf, comenzaremos **ingestando el documento** usando `PyPDFLoader` (pueden consultar más loaders disponibles en la siguiente [página](https://python.langchain.com/docs/integrations/document_loaders/)):

In [11]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false" # path al documento
loader = PyPDFLoader(file_path) # inicializar loader de PDF

docs = loader.load() # cargar documento
docs

[Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 0}, page_content='Elecciones Municipales 2024\nwww.decidechile.cl\nAnálisis de los resultados de los comicios del \n26 y 27 de octubre 2024 en Chile.\nOCTUBRE, 2024'),
 Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 1}, page_content='#MUNICIPALES2024\nINFORME IV -MUNICIPALES 2024\n•Alcaldes-Resultados-Los más y los menos votados-Financiamiento-Análisis:-Género y rango etario-Población gobernada-Comunas gobernadas-Transición de votos-General\n2INTRODUCCIÓN> GOBERNADORES > ALCALDES > CNCEJALES > CONSEJEROS > EL DÍA DESPUÉS\nContenido•Introducción-Participación-Análisis-Oficialismo-Oposición-Candidatos•Gobernadores-Resultados-Los más y los menos votados-Financiamiento-Análisis:-Género y rango etario-General\n•Concejales-Resultados-Los más y los menos votado

In [12]:
len(docs) # noten como se genera una lista de 55 elementos, en este caso cada elemento es una página

55

In [13]:
docs[0] # cada elemento es del tipo Document, el cual posee el contenido de cada página

Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 0}, page_content='Elecciones Municipales 2024\nwww.decidechile.cl\nAnálisis de los resultados de los comicios del \n26 y 27 de octubre 2024 en Chile.\nOCTUBRE, 2024')

#### **Split**

Después de haber ingestado el documento, el siguiente paso es dividir cada documento en chunks de texto más pequeños. Para eso usaremos `RecursiveCharacterTextSplitter` (nuevamente, pueden consultar otros tipos de splitters en el siguiente [link](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)):

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs) # dividir documentos en chunks
splits[:5]

[Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 0}, page_content='Elecciones Municipales 2024\nwww.decidechile.cl\nAnálisis de los resultados de los comicios del \n26 y 27 de octubre 2024 en Chile.\nOCTUBRE, 2024'),
 Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 1}, page_content='#MUNICIPALES2024\nINFORME IV -MUNICIPALES 2024\n•Alcaldes-Resultados-Los más y los menos votados-Financiamiento-Análisis:-Género y rango etario-Población gobernada-Comunas gobernadas-Transición de votos-General\n2INTRODUCCIÓN> GOBERNADORES > ALCALDES > CNCEJALES > CONSEJEROS > EL DÍA DESPUÉS\nContenido•Introducción-Participación-Análisis-Oficialismo-Oposición-Candidatos•Gobernadores-Resultados-Los más y los menos votados-Financiamiento-Análisis:-Género y rango etario-General'),
 Document(metadata={'source': 'https://gitlab.c

In [15]:
splits[0] # cada elemento es un Document, esta vez con menos contenido que en el paso anterior

Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 0}, page_content='Elecciones Municipales 2024\nwww.decidechile.cl\nAnálisis de los resultados de los comicios del \n26 y 27 de octubre 2024 en Chile.\nOCTUBRE, 2024')

In [16]:
len(splits) # noten como ahora contamos con 162 chunks (pues 1 página contiene más de 1 chunk)

162

#### **Embed & Store**

Con los documentos separados en chunks, pasamos ahora a transformarlos a *embeddings*. Para esto simplemente ocupamos embeddings sentence-transformers/all-mpnet-base-v2 través de `HuggingFace` (de nuevo, pueden revisar más opciones de embeddings en la siguiente [página](https://python.langchain.com/v0.1/docs/integrations/text_embedding/)).

Por el lado del vector store, usaremos `FAISS` (nuevamente, pueden revisar más opciones en este [link](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)).

In [17]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

vectorstore = FAISS.from_documents(splits, embeddings)
vectorstore

<ipython-input-17-64c2525233a8>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
vectorstore

### **Retrieval and Generation**

Con los documentos indexados, el siguiente paso es **habilitar que nuestro LLM pueda conseguir la información de los documentos** para responder preguntas.

#### **Retrieve**

Para esto, el primer paso es habilitar nuestro *vector store* para buscar los documentos más relevantes. Para esto, simplemente usamos el método `.as_retriever`:

In [19]:
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f01666e3f10>, search_kwargs={'k': 3})

Con esto, podemos probar nuestro retriever:

In [20]:
question = "como fue la participacion en las elecciones?" # pregunta
relevant_documents = retriever.invoke(question) # top k documentos relevantes a la pregunta
relevant_documents

[Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 4}, page_content='alta participación en comparación con elecciones anteriores bajo el sistema de voto voluntario y es levemente superior incluso a eventos recientes con voto obligatorio, consolidando la efectividad de estas medidas en aumentar la participación electoral.'),
 Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 4}, page_content='Evolución de la ParticipaciónElecciones 1989 –2024\nINTRODUCCIÓN> GOBERNADORES > ALCALDES > CONCEJALES > CONSEJEROS\nFuente: Este gráfico se ha elaborado con datos proporcionados por el Servel.\n@DecideChile'),
 Document(metadata={'source': 'https://gitlab.com/sebatinoco/datos/-/raw/main/Informe-DecideChile-Municipales-2024.pdf?inline=false', 'page': 4}, page_content='•La participación en las elecciones municipales de 2

Noten como la información relevante está almaenada en el atributo **page_content**:

In [21]:
relevant_documents[0].page_content

'alta participación en comparación con elecciones anteriores bajo el sistema de voto voluntario y es levemente superior incluso a eventos recientes con voto obligatorio, consolidando la efectividad de estas medidas en aumentar la participación electoral.'

Como solo nos interesa ese atributo, podemos definir la función `format_docs` para procesar los chunks:

In [22]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

print(format_docs(relevant_documents))

alta participación en comparación con elecciones anteriores bajo el sistema de voto voluntario y es levemente superior incluso a eventos recientes con voto obligatorio, consolidando la efectividad de estas medidas en aumentar la participación electoral.

Evolución de la ParticipaciónElecciones 1989 –2024
INTRODUCCIÓN> GOBERNADORES > ALCALDES > CONCEJALES > CONSEJEROS
Fuente: Este gráfico se ha elaborado con datos proporcionados por el Servel.
@DecideChile

•La participación en las elecciones municipales de 2024 llegó a un notable 84,9%, convirtiéndose en una de las más altas desde 1989. No se veía un nivel de participación tan elevado en comicios municipales desde 1992.•Este aumento considerable es atribuible principalmente al voto obligatorio y la inscripción automática, que, aunque implementados previamente, exhibieron su impacto notoriamente también en esta elección. •Esta cifra muestra una alta participación en comparación con elecciones


Finalmente, `LangChain` nos permite unir todos estos pasos en una **chain**:

In [23]:
retriever_chain = retriever | format_docs # chain
print(retriever_chain.invoke("como fue la participacion en las elecciones?"))

alta participación en comparación con elecciones anteriores bajo el sistema de voto voluntario y es levemente superior incluso a eventos recientes con voto obligatorio, consolidando la efectividad de estas medidas en aumentar la participación electoral.

Evolución de la ParticipaciónElecciones 1989 –2024
INTRODUCCIÓN> GOBERNADORES > ALCALDES > CONCEJALES > CONSEJEROS
Fuente: Este gráfico se ha elaborado con datos proporcionados por el Servel.
@DecideChile

•La participación en las elecciones municipales de 2024 llegó a un notable 84,9%, convirtiéndose en una de las más altas desde 1989. No se veía un nivel de participación tan elevado en comicios municipales desde 1992.•Este aumento considerable es atribuible principalmente al voto obligatorio y la inscripción automática, que, aunque implementados previamente, exhibieron su impacto notoriamente también en esta elección. •Esta cifra muestra una alta participación en comparación con elecciones


#### **Generation**

Con el retriever ya listo, la única pieza faltante es **entregar los documentos relevantes al LLM** para que pueda responder de mejor manera.

Para esto, primero definiremos un **template** para con las instrucciones a seguir:

In [24]:
from langchain_core.prompts import PromptTemplate

# noten como ahora existe el parámetro de context!
rag_template = '''
Eres un asistente experto en la interpretación de resultados electorales de la política chilena.
Tu único rol es contestar preguntas del usuario a partir de información relevante que te sea proporcionada.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante: {context}
Pregunta: {question}
Respuesta útil:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

Finalmente, condensamos todo en una chain para recuperar información relevante y responder al mismo tiempo:

In [25]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

question = "que información tienes sobre nulos y blancos?"
response = rag_chain.invoke(question)
print(response)

La información disponible indica que entre un 16% y un 18% del total de votos fueron nulos o blancos.  Este porcentaje es mayor en jóvenes que en otros grupos etarios.

Específicamente:

* **Por género:**  En hombres, el porcentaje de votos nulos o blancos fue de 17,1%, mientras que en mujeres fue de 13,3%.  En ambos casos, esto representa una disminución respecto a la elección de mayo de 2023.

* **Por grupo etario:**  En el grupo de 18 a 34 años, el porcentaje de votos nulos o blancos fue de 19% en hombres y 17% en mujeres.  En mujeres mayores de 34 años, este porcentaje estuvo entre 11% y 12%, mientras que en hombres mayores de 34 años, entre 16% y 18%.

* **Por elección:** En la elección de Consejeros Regionales, una de cada cuatro papeletas (25%) no marcó una preferencia clara.  En la elección de Concejales, uno de cada cinco votos (20%) fueron nulos o blancos, un aumento significativo respecto al 5,74% de 2021.  En la elección de Alcaldes, hubo un aumento proporcional del 460% en

## **Agentes 🕵️‍♂️**

Ahora que ya conocemos cómo implementar un RAG básico sobre documentos PDF, buscaremos hacer lo mismo (es decir, nutrir nuestro LLM de fuentes externas) pero con dos diferencias:

- Alimentaremos nuestro agente de la información proporcionada por **motores de búsqueda** (e.g: Google).
- Remplazaremos las chains por **Agentes**.

Todo genial, pero...

### **¿Qué son los Agentes?**

Un agente se define como un modelo que tiene la capacidad para ejecutar **acciones**. De esta manera, el objetivo del agente es **elegir una <u>secuencia</u> de acciones** a realizar para cumplir con un objetivo específico.

Los **agentes** utilizan el **LLM como <u>motor de razonamiento</u>** para determinar **qué acciones tomar y en qué orden**.

Ahora que ya conocemos qué es un Agente, conozcamos uno de los framework más famosos para implementar Agentes con LLM.

<center>
<img src='https://media4.giphy.com/media/dBZsIa2eWkVBaU6lWY/giphy.gif' width=450/>
</center>

### **ReAct**

Introducido en el paper de [Yao et al. (2022)](https://arxiv.org/abs/2210.03629), **ReAct** se presenta como un framework para que agentes puedan **razonar sobre acciones** a partir de observaciones. En particular, para cumplir un objetivo un agente basado en ReAct debe seguir la siguiente secuencia:

- El agente **razona** sobre qué acción tomar.
- En base al razonamiento hecho, el **agente ejecuta la acción**.
- A partir de la acción ejecutada, el **agente observa y evalúa el nuevo escenario** (feedback).



<center>
<img src='https://peterroelants.github.io/images/llm/ReAct_loop.png' width=400/>
</center>

Por último, es importante señalar que Los resultados muestran como los **agentes basados en ReAct** puede **superar múltiples benchmarks en tareas de lenguaje y toma de decisiones**, además de mejorar la interpretabiliad y confiabilidad en los LLM.

Pongamos en práctica lo aprendido con un ejemplo!

Comencemos primero cargando un prompt predefinido del **hub** de langchain para usar ReAct:

In [26]:
from langchain import hub

react_prompt = hub.pull("hwchase17/react") # template de ReAct
print(react_prompt.template)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


> **Pregunta:** Tómense un momento para estudiar el prompt. ¿Qué variables recibe?

Una de las ventajas de usar agentes es que tienen una fácil integración con **tools**, es decir, **herramientas que puede usar el agente para lograr un objetivo** en particular.

Para este caso particular, usaremos la tool del motor de búsqueda `Tavily` para permitir que nuestro agente pueda recuperar **información de la web** (pueden consultar más tools en el siguiente [link](https://api.python.langchain.com/en/v0.1/community_api_reference.html#module-langchain_community.tools)):

In [27]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 1) # inicializamos tool
tools = [search] # guardamos las tools en una lista

Con las tools definidas, podemos inicializar nuestro agente ReAct:

In [28]:
from langchain.agents import create_react_agent, AgentExecutor

agent = create_react_agent(llm, tools, react_prompt) # primero inicializamos el agente ReAct
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # lo transformamos a AgentExecutor para habilitar la ejecución de tools
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'tool_names': 'tavily_search_results_json'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: 

Finalmente, probamos nuestro agente:

In [29]:
response = agent_executor.invoke({"input": "Dónde habitan los pingüinos y en qué se basa su alimentación?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: To answer this question, I need information about penguin habitats and diets.  I'll use the search engine to find this information.

Action: tavily_search_results_json
Action Input: "penguin habitat and diet"
[{'url': 'https://birdfact.com/articles/what-do-penguins-eat', 'content': 'Unlike other penguin species, and due to its habitat, the Galápagos Penguin enjoys a diet rich in small schooling fish. Native to the Galápagos Islands, these small penguins forage in tropical waters, teeming with cold water fish species, including anchovies, sardines and mullet, which thrive in the waters cooled by the Humboldt Current.'}]Thought: The observation provides information about the diet of one penguin species, but not a comprehensive overview of all penguin habitats and diets. I need a broader search.

Action: tavily_search_results_json
Action Input: "penguin habitat and diet all species"
[{'url': 'https://birdfact.com/articles/what-do-penguins-e

### **Implementando nuestras propias tools**

Algo interesante que podemos hacer es **programar nuestras propias tools** para que el agente interactúe con ellas.

Revisemos un ejemplo en que programos tools con algunas **operaciones matemáticas**:

<center>
<img src='https://media1.tenor.com/images/dfe0c1c6eaf41b91996aacee0879ebc2/tenor.gif?itemid=3486402' width=400  />
</center>

In [30]:
from langchain.tools import tool

@tool
def multiply(x: int or float, y: int or float) -> float:
    """Multiply 'x' times 'y'."""
    return float(x * y)

@tool
def exponentiate(x: int or float, y: int or float) -> float:
    """Raise 'x' to the 'y'."""
    return float(x**y)

@tool
def add(x: int or float, y: int or float) -> float:
    """Add 'x' and 'y'."""
    return float(x + y)

Luego, simplemente agrupamos las tools en una lista:

In [31]:
tools = [add, multiply, exponentiate]

En paralelo, crearemos un **prompt** para nuestro agente:

In [32]:
# noten como ahora se incluye la variable agent_scratchpad
math_template = """
Eres un asistente experto en matemáticas.
Tu único rol es responder la pregunta del usuario usando las tools disponibles.

Pregunta: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(math_template)
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='\nEres un asistente experto en matemáticas.\nTu único rol es responder la pregunta del usuario usando las tools disponibles.\n\nPregunta: {input}\n{agent_scratchpad}\n')

Con el prompt creado, pasamos a **crear nuestro agente**.

Noten que como nuestras tools reciben más de un parámetro de entrada (a y b), **remplazaremos ReAct por [Tool Calling](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)** (de igual manera, pueden encontrar todos los tipos de Agentes disponibles y sus limitantes en el siguiente [link](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)):

In [33]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)
math_agent = AgentExecutor(agent=agent, tools=tools, verbose=True)
math_agent

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='\nEres un asistente experto en matemáticas.\nTu único rol es responder la pregunta del usuario usando las tools disponibles.\n\nPregunta: {input}\n{agent_scratchpad}\n')
| RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f02710aab00>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': "Add 'x' and 'y'.", 'parameters': {'properties': {'x': {'type': 'integer'}, 'y': {'type': 'integer'}}, 'required': ['x', 'y'], 'type': 'object'}}}, 

Finalmente, podemos probar el funcionamiento de nuestro agente:

In [34]:
math_agent.invoke({"input": "cuanto es 10 ** 3 + 5 * 1.4?",})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'y': 3.0, 'x': 10.0}`


1000.0
Invoking: `multiply` with `{'y': 1.0, 'x': 5.0}`


5.0
Invoking: `exponentiate` with `{'y': 3.0, 'x': 10.0}`


1000.0
Invoking: `multiply` with `{'y': 1.0, 'x': 5.0}`


5.0
Invoking: `exponentiate` with `{'y': 3.0, 'x': 10.0}`


1000.0
Invoking: `multiply` with `{'y': 1.0, 'x': 5.0}`


5.0
Invoking: `exponentiate` with `{'y': 3.0, 'x': 10.0}`
responded: Primero, calculamos 10 elevado a la potencia de 3:



Esto resulta en 1000.

Luego, calculamos 5 multiplicado por 1.4:



Esto resulta en 5.

Finalmente, sumamos los dos resultados: 1000 + 5 = 1005.

Por lo tanto, la respuesta es $\boxed{1005}$


1000.0
Invoking: `multiply` with `{'y': 1.0, 'x': 5.0}`
responded: Primero, calculamos 10 elevado a la potencia de 3:



Esto resulta en 1000.

Luego, calculamos 5 multiplicado por 1.4:



Esto resulta en 5.

Finalmente, sumamos los dos resultados: 1000 + 5 = 1005.

Por lo tanto, la respuesta


Invoking: `exponentiate` with `{'y': 3.0, 'x': 10.0}`
responded: Primero, calculamos 10 elevado a la potencia de 3:



Esto resulta en 1000.

Luego, calculamos 5 multiplicado por 1.4:



Esto resulta en 5.

Finalmente, sumamos los dos resultados: 1000 + 5 = 1005.

Por lo tanto, la respuesta es $\boxed{1005}$


1000.0
Invoking: `multiply` with `{'y': 1.0, 'x': 5.0}`
responded: Primero, calculamos 10 elevado a la potencia de 3:



Esto resulta en 1000.

Luego, calculamos 5 multiplicado por 1.4:



Esto resulta en 5.

Finalmente, sumamos los dos resultados: 1000 + 5 = 1005.

Por lo tanto, la respuesta es $\boxed{1005}$


5.0


Invoking: `exponentiate` with `{'y': 3.0, 'x': 10.0}`
responded: Primero, calculamos 10 elevado a la potencia de 3:



Esto resulta en 1000.

Luego, calculamos 5 multiplicado por 1.4:



Esto resulta en 5.

Finalmente, sumamos los dos resultados: 1000 + 5 = 1005.

Por lo tanto, la respuesta es $\boxed{1005}$


1000.0
Invoking: `multiply` with `{'y': 1.0, 'x': 5.0}`
responded: Primero, calculamos 10 elevado a la potencia de 3:



Esto resulta en 1000.

Luego, calculamos 5 multiplicado por 1.4:



Esto resulta en 5.

Finalmente, sumamos los dos resultados: 1000 + 5 = 1005.

Por lo tanto, la respuesta es $\boxed{1005}$


5.0Primero, calculamos 10 elevado a la potencia de 3:

Esto resulta en 1000.

Luego, calculamos 5 multiplicado por 1.4:

Esto resulta en 7.

Finalmente, sumamos los dos resultados: 1000 + 7 = 1007.

Por lo tanto, la respuesta es $\boxed{1007}$


> Finished chain.


{'input': 'cuanto es 10 ** 3 + 5 * 1.4?',
 'output': 'Primero, calculamos 10 elevado a la potencia de 3:\n\nEsto resulta en 1000.\n\nLuego, calculamos 5 multiplicado por 1.4:\n\nEsto resulta en 7.\n\nFinalmente, sumamos los dos resultados: 1000 + 7 = 1007.\n\nPor lo tanto, la respuesta es $\\boxed{1007}$\n'}

## **Soluciones Multi Agente 👨‍👩‍👦‍👦**

<center>
<img src='https://media.tenor.com/FApRE_u99tgAAAAC/teamwork-team-game.gif' width=400  />
</center>

En las secciones pasadas habilitamos agentes que puedan hacer RAG sobre fuentes externas:

- Una **chain** que responde preguntas de las últimas elecciones municipales en base a un informe en PDF
- Un **agente** que responde preguntas matemáticas a partir de tools creadas manualmente.

Con esto en consideración, nace la pregunta natural: **¿Qué pasa si combinamos ambas soluciones?**

El objetivo de esta sección es introducirlos al paradigma **multiagente**, es decir, **combinar 2 o más funcionalidades en un mismo chat**. En particular, buscaremos implementar una arquitectura simple de enrutamiento, la cual consta de 4 agentes:

- **Agente router**, el cual recibe y dirige la pregunta del usuario a alguno de los agentes.
- **Agente de elecciones**: responde preguntas sobre las elecciones municipales 2024
- **Agente experto en matemáticas**: responde preguntas matemáticas
- **Agente de redireccionamiento**: en caso de que la pregunta del usuario no pertenezca a alguno de los temas anteriores, invita al usuario a reorientar su pregunta (esto es útil para evitar preguntas maliciosas).

**Nota**: Para efectos de esta sección y por simplicidad, no se hace distinción entre Agente y Chain.

<center>
<img src='https://preview.redd.it/smart-orchestrator-router-for-multiple-specialized-llms-v0-gjgkmlbu3jlc1.png?width=627&format=png&auto=webp&s=13ef701d45f642ce36ae8e99cb172b903fe7d36b' width=600  />
</center>

**Importante: La implementación es sólo ilustrativa y <u>no cumple con el estándar actual recomendado por LangChain</u>. Si desean conocer mejor cómo implementar soluciones multiagente, les recomiendo estudiar [LangGraph](https://www.langchain.com/langgraph)**.

#### **Agente Router**

Primero comenzamos creando nuestro agente router:

In [40]:
router_prompt = PromptTemplate.from_template(
    """
    Eres un asistente experto en la clasificación de preguntas del usuario.
    Tu único rol es clasificar preguntas del usuario en las categorías 'elecciones', 'math', u 'otro' según el siguiente criterio:
    - 'elecciones': Cuando la pregunta sea relacionada con las elecciones municipales de chile 2024.
    - 'math': Cuando la pregunta sea relacionada a preguntas de matemáticas
    - 'otro': Todo aquella pregunta que no esté contenida en las categorías anteriores.

    No respondas con más de una palabra y no incluyas.

    <pregunta>
    {question}
    </pregunta>

    Categoría:"""
)

router_chain = (
    router_prompt
    | llm
    | StrOutputParser()
)

router_chain.invoke({"question": "cuanto es 2+2"})

'math\n'

#### **Agente Redirect**

Repetimos lo mismo para crear nuestro agente de redireccionamiento:

In [41]:
redirect_prompt = PromptTemplate.from_template(
    """
    Eres un asistente experto en el redireccionamiento de preguntas de usuarios.
    Vas a recibir una pregunta del usuario, tu único rol es indicar que no puedes responder su pregunta y redireccionar al usuario
    para que te pregunte sobre las elecciones municipales de Chile 2024 o cálculos matemáticos.

    Recuerda ser amable y cordial en tu respuesta.

    Pregunta: {question}
    Respuesta cordial:"""
)

redirect_chain = (
    redirect_prompt
    | llm
    | StrOutputParser()
)

redirect_chain.invoke({"question": "dame la receta para hacer una pizza"})

'¡Hola!  Entiendo que quieres la receta para hacer una pizza, ¡qué rico!  Sin embargo, mi conocimiento especializado se centra en las elecciones municipales de Chile 2024 y en cálculos matemáticos.  Para obtener una deliciosa receta de pizza, te recomiendo buscar en internet, consultar un libro de cocina o preguntar en un foro de gastronomía.  Si tienes alguna pregunta sobre las elecciones municipales chilenas del 2024 o necesitas ayuda con algún cálculo matemático, ¡con mucho gusto te ayudaré!\n'

#### **Juntando todo**

Finalmente, podemos juntar todo lo que hemos desarrollado en una sola función:

In [42]:
def route_question(question):
  '''
  Recibe una pregunta de usuario.
  Rutea la pregunta al agente respectivo y responde de manera acorde.
  '''

  topic = router_chain.invoke({"question": question}) # enrutamiento

  if "elecciones" in topic: # si la pregunta es sobre las elecciones, utilizar cadena
      return rag_chain.invoke(question)
  elif "math" in topic: # si la pregunta es de matemáticas, utilizar agente
      return math_agent.invoke({"input": question})["output"]
  else: # de lo contrario, redireccionar pregunta
      return redirect_chain.invoke({"question": question})

Para finalmente hacer pruebas de su funcionamiento:

In [43]:
print(route_question("cómo puedo hacerme millonario?"))

¡Hola!  Entiendo que quieres saber cómo hacerte millonario, ¡una meta admirable!  Sin embargo, esa es una pregunta bastante amplia que va más allá de mis capacidades como asistente.  No puedo ofrecerte consejos financieros específicos.

Para poder ayudarte mejor, me gustaría enfocar nuestra conversación en temas donde sí puedo ser de utilidad.  ¿Te interesaría, por ejemplo, hablar sobre las elecciones municipales de Chile en 2024, o quizás prefieres que te ayude con algún cálculo matemático?  Dime qué te parece y con gusto te ayudaré en lo que pueda dentro de esos ámbitos.



In [44]:
print(route_question("como fue la participación de las elecciones?"))

La participación en las elecciones municipales de 2024 alcanzó un 84,9%, una de las más altas desde 1989 y superior incluso a elecciones recientes con voto obligatorio.  Este alto nivel de participación se atribuye al voto obligatorio y la inscripción automática, mostrando la efectividad de estas medidas para aumentar la participación electoral.  La participación fue considerablemente mayor que en elecciones anteriores bajo el sistema de voto voluntario.



In [45]:
print(route_question("cuanto es 5+3?"))



> Entering new AgentExecutor chain...

Invoking: `add` with `{'y': 3.0, 'x': 5.0}`


8.08.0


> Finished chain.
8.0

